<a href="https://colab.research.google.com/github/natmarqs/PSC---PEE/blob/main/mvp_p1_natalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Base de dados pública referente ao desmatamento dos biomas brasileiros, disponível em: https://basedosdados.org/dataset/e5c87240-ecce-4856-97c5-e6b84984bf42?table=d7a76d45-c363-4494-826d-1580e997ebf0

In [1]:
import pandas as pd, numpy as np, requests, io, zipfile, json, re, os, math, csv
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import pandas as pd
from IPython.display import display

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV # Para otimização

Mounted at /content/gdrive


# Ler o dataset

In [2]:
from google.colab import drive
drive.mount

#pasta
dataset_path ='/content/gdrive/MyDrive/P1_Dados'

file_name ='br_inpe_prodes_municipio_bioma.csv'

caminho_completo = os.path.join(dataset_path, file_name)

# Ler o arquivo
df1 = pd.read_csv(caminho_completo)

n_row1 = df1.shape[0]
print("Número de registro de desmatamento:", n_row1)
df1

Número de registro de desmatamento: 156864


,ano,id_municipio,bioma,area_total,desmatado,vegetacao_natural,nao_vegetacao_natural,hidrografia
0,2009,5102504,Pantanal,17422.29079,3124.8,12616.5,0.4,1680.6
1,2011,5102504,Pantanal,17422.29079,3203.4,12537.9,0.4,1680.6
2,2010,5102504,Pantanal,17422.29079,3157.8,12583.5,0.4,1680.6
3,2005,5102504,Pantanal,17422.29079,2937.1,12804.2,0.4,1680.6
4,2002,5102504,Pantanal,17422.29079,2598.0,13143.3,0.4,1680.6
...,...,...,...,...,...,...,...,...
156859,2015,4306551,Mata Atlântica,78.29205,44.5,18.7,0.0,15.1
156860,2005,4306551,Mata Atlântica,78.29205,44.0,19.2,0.0,15.1
156861,2019,4306551,Mata Atlântica,78.29205,44.9,18.3,0.0,15.1
156862,2004,4306551,Mata Atlântica,78.29205,44.0,19.2,0.0,15.1


# Organização do dado

In [3]:
# Selecionar colunas e garantir que 'ano' é um inteiro
df_series = df1[['ano', 'bioma', 'desmatado']].copy()
df_series['ano'] = df_series['ano'].astype(int)

# Agrupar e somar a área desmatada por Bioma e Ano
df_agregado = df_series.groupby(['bioma', 'ano'])['desmatado'].sum().reset_index()

# Renomear a coluna para definir as unidades de medida (CORREÇÃO DE FLUXO)
df_agregado.rename(columns={'desmatado': 'desmatado_anual_total_km2'}, inplace=True)

# Assumi-se que o dado de desmatamento de um ano é medido no final daquele ano.
df_agregado['Data'] = pd.to_datetime(df_agregado['ano'], format='%Y')


# Ordenar o DataFrame!
# Ordem: 1º Bioma (alfabética), 2º Ano (crescente: 2000, 2001, 2002...)
df_agregado_ordenado = df_agregado.sort_values(by=['bioma', 'ano'], ascending=True)

print("--- Dados Históricos de Desmatamento Agregado (Organizado por Bioma e Ano) ---")
print(df_agregado_ordenado[['bioma', 'ano', 'desmatado_anual_total_km2']])

# Divisão Temporal
df_agregado = df_agregado_ordenado.copy()

# Encontrar o ano inicial e final no dataset
ano_minimo = df_agregado['ano'].min()
ano_maximo = df_agregado['ano'].max()

# Definir o número de anos para o conjunto de teste (ex: 3 anos)
ANOS_TESTE = 3

# Definir o ano de corte: tudo antes ou igual a este ano é TREINO
ano_corte = ano_maximo - ANOS_TESTE

print(f"Total de Anos no Dataset: {ano_maximo - ano_minimo + 1}")
print(f"Ano Mínimo: {ano_minimo}")
print(f"Ano Máximo: {ano_maximo}")
print(f"--------------------------------------------------")
print(f"O conjunto de TREINO irá até o ano: {ano_corte}")
print(f"O conjunto de TESTE incluirá os anos: {ano_corte + 1} a {ano_maximo}")

# === LINHAS ADICIONADAS PARA DEFINIR AS VARIÁVEIS df_treino E df_teste ===
# 1. Conjunto de Treino
df_treino = df_agregado[df_agregado['ano'] <= ano_corte]

# 2. Conjunto de Teste
df_teste = df_agregado[df_agregado['ano'] > ano_corte]

print(f"Dimensão do Treino: {df_treino.shape[0]} linhas")
print(f"Dimensão do Teste: {df_teste.shape[0]} linhas")

--- Dados Históricos de Desmatamento Agregado (Organizado por Bioma e Ano) ---
        bioma   ano  desmatado_anual_total_km2
0    Amazônia  2000                   474934.6
1    Amazônia  2001                   527870.0
2    Amazônia  2002                   552402.0
3    Amazônia  2003                   581385.6
4    Amazônia  2004                   607431.0
..        ...   ...                        ...
139  Pantanal  2019                    27377.1
140  Pantanal  2020                    28054.8
141  Pantanal  2021                    28879.2
142  Pantanal  2022                    29668.5
143  Pantanal  2023                    30391.7

[144 rows x 3 columns]
Total de Anos no Dataset: 24
Ano Mínimo: 2000
Ano Máximo: 2023
--------------------------------------------------
O conjunto de TREINO irá até o ano: 2020
O conjunto de TESTE incluirá os anos: 2021 a 2023
Dimensão do Treino: 126 linhas
Dimensão do Teste: 18 linhas


# Treino e Teste

In [4]:
# 1. Conjunto de Treino
df_treino = df_agregado[df_agregado['ano'] <= ano_corte]

# 2. Conjunto de Teste
df_teste = df_agregado[df_agregado['ano'] > ano_corte]

# --- Impressão e Verificação ---
print(f"Total de Anos no Dataset: {ano_maximo - ano_minimo + 1}")
print(f"--------------------------------------------------")
print(f"O conjunto de TREINO irá até o ano: {ano_corte}")
print(f"O conjunto de TESTE incluirá os anos: {ano_corte + 1} a {ano_maximo}")

print(f"\n--- Estrutura dos Conjuntos ---")
print(f"Dimensão do Treino: {df_treino.shape[0]} linhas")
print(f"Dimensão do Teste: {df_teste.shape[0]} linhas")

# Exemplo de anos no Treino (por bioma)
print("\nExemplo de anos no TREINO (Amazônia):",
      df_treino[df_treino['bioma'] == 'Amazônia']['ano'].unique())
print("Exemplo de anos no TESTE (Amazônia):",
      df_teste[df_teste['bioma'] == 'Amazônia']['ano'].unique())

Total de Anos no Dataset: 24
--------------------------------------------------
O conjunto de TREINO irá até o ano: 2020
O conjunto de TESTE incluirá os anos: 2021 a 2023

--- Estrutura dos Conjuntos ---
Dimensão do Treino: 126 linhas
Dimensão do Teste: 18 linhas

Exemplo de anos no TREINO (Amazônia): [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020]
Exemplo de anos no TESTE (Amazônia): [2021 2022 2023]


# Definição do Modelo Baseline (Naive Forecast)

O Modelo Naive prevê que o valor do próximo período será exatamente igual ao valor do período anterior.

Previsão
(Ano X)
​
 =Valor Real
(Ano X - 1)
​

Prevê que o desmatamento de um ano no conjunto de teste será igual ao desmatamento do último ano no conjunto de treino.

In [5]:
# A coluna de desmatamento é 'desmatado_anual_total_km2'
TARGET_COL = 'desmatado_anual_total_km2'

# DataFrame para armazenar as previsões do Baseline
df_baseline_preds = pd.DataFrame()

# Iterar sobre cada bioma
for bioma_nome, df_treino_bioma in df_treino.groupby('bioma'):
    # Encontra o último valor de desmatamento no conjunto de TREINO para este bioma
    ultimo_valor_treino = df_treino_bioma.sort_values(by='ano', ascending=False).iloc[0][TARGET_COL]

    # Prepara o DataFrame de teste para este bioma
    df_teste_bioma = df_teste[df_teste['bioma'] == bioma_nome].copy()

    # A previsão do baseline para CADA linha do teste é o último valor de treino
    df_teste_bioma['baseline_pred'] = ultimo_valor_treino

    # Acumula os resultados
    df_baseline_preds = pd.concat([df_baseline_preds, df_teste_bioma])

# Garantir a ordenação
df_baseline_preds = df_baseline_preds.sort_values(by=['bioma', 'ano'])

print("Modelo Baseline (Naive) Gerado.")

Modelo Baseline (Naive) Gerado.


# Cálculo das Métricas de Avaliação

Agora, calcula-se as métricas de performance para o Baseline, usado como referência para todos os modelos futuros.

Métricas padrão para problemas de Regressão:

RMSE (Root Mean Squared Error): Penaliza erros grandes.

MAE (Mean Absolute Error): Média simples dos erros absolutos.

R^2 =  (Coeficiente de Determinação): Mede o quão bem o modelo explica a variância.

In [6]:
#métricas de performance para o Baseline, que usada como referência para todos os modelos futuros.

# Valores Reais no Conjunto de Teste
y_true = df_baseline_preds[TARGET_COL]

# Previsões do Baseline
y_pred_baseline = df_baseline_preds['baseline_pred']

# Cálculo das Métricas
rmse_baseline = np.sqrt(mean_squared_error(y_true, y_pred_baseline))
mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
r2_baseline = r2_score(y_true, y_pred_baseline)

print("\n" + "="*50)
print("     MÉTRICAS DO MODELO BASELINE (NAIVE)     ")
print("="*50)
print(f"RMSE (km²): {rmse_baseline:.2f}")
print(f"MAE (km²):  {mae_baseline:.2f}")
print(f"R² Score:   {r2_baseline:.4f}")
print("="*50)


     MÉTRICAS DO MODELO BASELINE (NAIVE)     
RMSE (km²): 13529.54
MAE (km²):  8840.93
R² Score:   0.9986


##Detalhes do Cálculo da Previsão Linear com Regressão

A previsão da área desmatada anual para os próximos anos foi realizada utilizando o modelo de Regressão Linear Simples, que é ideal para projetar dados que seguem uma tendência constante no tempo.

1. Modelo de Regressão Linear
O modelo estabelece uma relação linear de causa e efeito, onde a variável de tempo (Ano) é usada para prever a variável de interesse (Área Desmatada).

A fórmula matemática utilizada é:

**y=a+bx**

Onde:

x =	ano

y =	desmatado_anual_total_km^2

b (Coeficiente Angular ou Declividade): É a taxa de mudança. Ele representa o aumento (ou diminuição) médio do desmatamento (em km
2) para cada aumento de um ano no tempo.

a (Coeficiente Linear ou Intercepto): É o valor de y quando x é zero.

In [7]:
TARGET_COL = 'desmatado_anual_total_km2'
FEATURE_COL = 'ano'


df_regressao_preds = pd.DataFrame()


print("Iniciando treinamento e avaliação do Modelo de Regressão Linear...")

# Iterar sobre cada bioma (a Regressão é treinada por bioma)
for bioma_nome in df_treino['bioma'].unique():

    # Filtrar Treino e Teste para o bioma atual
    df_treino_bioma = df_treino[df_treino['bioma'] == bioma_nome]
    df_teste_bioma = df_teste[df_teste['bioma'] == bioma_nome].copy()

    # 1. Preparar os dados para o Scikit-learn (X deve ser 2D)
    X_treino = df_treino_bioma[FEATURE_COL].values.reshape(-1, 1)
    y_treino = df_treino_bioma[TARGET_COL].values

    X_teste = df_teste_bioma[FEATURE_COL].values.reshape(-1, 1)

    # 2. Treinar o modelo
    modelo_rl = LinearRegression()
    modelo_rl.fit(X_treino, y_treino)

    # 3. Fazer a Previsão no conjunto de TESTE
    previsoes_teste = modelo_rl.predict(X_teste)

    # 4. Adicionar as previsões ao DataFrame de teste
    df_teste_bioma['regressao_pred'] = previsoes_teste.clip(min=0) # Garante que o desmatamento não seja negativo

    # 5. Acumular os resultados
    df_regressao_preds = pd.concat([df_regressao_preds, df_teste_bioma])

print("Previsões de Regressão Linear concluídas no conjunto de TESTE.")

Iniciando treinamento e avaliação do Modelo de Regressão Linear...
Previsões de Regressão Linear concluídas no conjunto de TESTE.


In [8]:
# --- Avaliação e Comparação ---

# Valores Reais no Conjunto de Teste
y_true = df_regressao_preds[TARGET_COL]

# Previsões da Regressão Linear
y_pred_regressao = df_regressao_preds['regressao_pred']

# Cálculo das Métricas da Regressão Linear
rmse_regressao = np.sqrt(mean_squared_error(y_true, y_pred_regressao))
mae_regressao = mean_absolute_error(y_true, y_pred_regressao)
r2_regressao = r2_score(y_true, y_pred_regressao)


# --- Relatório Final de Comparação ---
print("\n" + "="*70)
print("              COMPARAÇÃO DE MODELOS NO CONJUNTO DE TESTE              ")
print("="*70)

dados_comparacao = {
    'Métrica': ['RMSE (km²)', 'MAE (km²)', 'R² Score'],
    # As variáveis de Baseline (rmse_baseline, etc.) vêm do bloco anterior.
    'Baseline (Ingênuo)': [rmse_baseline, mae_baseline, r2_baseline],
    'Regressão Linear': [rmse_regressao, mae_regressao, r2_regressao]
}

df_comparacao = pd.DataFrame(dados_comparacao)
df_comparacao = df_comparacao.set_index('Métrica').round(4)
display(df_comparacao)

print("\n" + "="*70)


              COMPARAÇÃO DE MODELOS NO CONJUNTO DE TESTE              


,Baseline (Ingênuo),Regressão Linear
Métrica,,
RMSE (km²),13529.5380,17671.0368
MAE (km²),8840.9333,14019.2924
R² Score,0.9986,0.9977


# Avaliação e comparação:

In [9]:
# --- Salvar tabela ---
# arquivo de saída
nome_arquivo_saida = "previsoes_teste_rl.csv"
caminho_pasta = "/content/gdrive/MyDrive/P1_Dados/"

# Caminho completo
caminho_completo_saida = caminho_pasta + nome_arquivo_saida

# CORREÇÃO: Usamos o nome correto do DataFrame (df_regressao_preds)
df_regressao_preds.to_csv(caminho_completo_saida, index=False, sep=';', decimal=',')

print("\n")
print("="*60)
print(f"✅ Tabela de previsões salva com sucesso no Google Drive:")
print(f"   Caminho: {caminho_completo_saida}")
print("="*60)



✅ Tabela de previsões salva com sucesso no Google Drive:
   Caminho: /content/gdrive/MyDrive/P1_Dados/previsoes_teste_rl.csv


# Implementação do Random Forest

In [10]:

TARGET_COL = 'desmatado_anual_total_km2'
FEATURE_COL = 'ano'

# DataFrame para armazenar as previsões do Random Forest no TESTE
df_rf_preds = pd.DataFrame()

print("\n" + "="*70)
print("Iniciando treinamento e avaliação do Modelo Random Forest (Não Otimizado)...")
print("="*70)

# Iterar sobre cada bioma
for bioma_nome in df_treino['bioma'].unique():

    # Filtrar Treino e Teste para o bioma atual
    df_treino_bioma = df_treino[df_treino['bioma'] == bioma_nome]
    df_teste_bioma = df_teste[df_teste['bioma'] == bioma_nome].copy()

    # Preparar os dados (X deve ser 2D)
    X_treino = df_treino_bioma[FEATURE_COL].values.reshape(-1, 1)
    y_treino = df_treino_bioma[TARGET_COL].values

    X_teste = df_teste_bioma[FEATURE_COL].values.reshape(-1, 1)

    # Treinar o modelo Random Forest com parâmetros iniciais
    modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)
    modelo_rf.fit(X_treino, y_treino)

    # Fazer a Previsão no conjunto de TESTE
    previsoes_teste = modelo_rf.predict(X_teste)

    # Adicionar as previsões ao DataFrame de teste
    df_teste_bioma['rf_pred'] = previsoes_teste.clip(min=0)

    # Acumular os resultados
    df_rf_preds = pd.concat([df_rf_preds, df_teste_bioma])

print("Previsões de Random Forest concluídas no conjunto de TESTE.")

# --- Cálculo das Métricas do Random Forest ---
y_true = df_rf_preds[TARGET_COL]
y_pred_rf = df_rf_preds['rf_pred']

rmse_rf = np.sqrt(mean_squared_error(y_true, y_pred_rf))
mae_rf = mean_absolute_error(y_true, y_pred_rf)
r2_rf = r2_score(y_true, y_pred_rf)

print(f"\nRMSE Random Forest (Inicial): {rmse_rf:.2f}")

# Exibir a tabela de comparação atual (Baseline, RL, RF Inicial)
dados_comparacao_atual = {
    'Métrica': ['RMSE (km²)', 'MAE (km²)', 'R² Score'],
    'Baseline (Ingênuo)': [rmse_baseline, mae_baseline, r2_baseline],
    'Regressão Linear': [rmse_regressao, mae_regressao, r2_regressao],
    'Random Forest (Inicial)': [rmse_rf, mae_rf, r2_rf]
}

df_comparacao_atual = pd.DataFrame(dados_comparacao_atual)
df_comparacao_atual = df_comparacao_atual.set_index('Métrica').round(4)
print("\n" + "="*70)
print("COMPARAÇÃO ATUAL")
display(df_comparacao_atual)
print("="*70)


Iniciando treinamento e avaliação do Modelo Random Forest (Não Otimizado)...
Previsões de Random Forest concluídas no conjunto de TESTE.

RMSE Random Forest (Inicial): 16406.89

COMPARAÇÃO ATUAL


,Baseline (Ingênuo),Regressão Linear,Random Forest (Inicial)
Métrica,,,
RMSE (km²),13529.5380,17671.0368,16406.8936
MAE (km²),8840.9333,14019.2924,11025.1049
R² Score,0.9986,0.9977,0.9980


# Otimização do Random Forest (Grid Search)

In [11]:
TARGET_COL = 'desmatado_anual_total_km2'
FEATURE_COL = 'ano'

# DataFrame para armazenar as previsões do modelo OTIMIZADO no TESTE
df_rf_otimizado_preds = pd.DataFrame()
df_best_params = [] # Lista para armazenar os melhores parâmetros encontrados por bioma

print("="*80)
print("Iniciando Otimização de Hiperparâmetros (GridSearch) no Random Forest...")
print("="*80)

# Definir a grade de hiperparâmetros a ser testada
# Esta grade é um bom ponto de partida, mas pode ser expandida se o tempo permitir.
param_grid = {
    'n_estimators': [50, 100, 200], # Número de árvores
    'max_depth': [3, 5, 8, 12],     # Profundidade máxima da árvore (Controla o Overfitting!)
    'random_state': [42]
}

# Iterar sobre cada bioma (a otimização é feita de forma independente)
for bioma_nome in df_treino['bioma'].unique():

    # Filtrar Treino e Teste para o bioma atual
    df_treino_bioma = df_treino[df_treino['bioma'] == bioma_nome]
    df_teste_bioma = df_teste[df_teste['bioma'] == bioma_nome].copy()

    # 1. Preparar os dados para o Grid Search
    X_treino = df_treino_bioma[FEATURE_COL].values.reshape(-1, 1)
    y_treino = df_treino_bioma[TARGET_COL].values

    X_teste = df_teste_bioma[FEATURE_COL].values.reshape(-1, 1)

    # Configurar o Grid Search (usamos neg_mean_squared_error para minimizar o erro)
    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=3, # Validação Cruzada de 3 dobras
        verbose=0,
        n_jobs=-1 # Usa todos os núcleos
    )

    # Executar o Grid Search (Treinamento e Otimização)
    grid_search.fit(X_treino, y_treino)

    # Salvar os Melhores Parâmetros
    df_best_params.append({
        'bioma': bioma_nome,
        'melhores_params': grid_search.best_params_
    })

    # Fazer a Previsão no conjunto de TESTE com o MELHOR MODELO ENCONTRADO
    melhor_modelo_rf = grid_search.best_estimator_
    previsoes_teste = melhor_modelo_rf.predict(X_teste)

    # Acumular os resultados
    df_teste_bioma['rf_otimizado_pred'] = previsoes_teste.clip(min=0)
    df_rf_otimizado_preds = pd.concat([df_rf_otimizado_preds, df_teste_bioma])

    print(f"  ✅ Otimização e previsão concluída para: {bioma_nome} | Best Params: {grid_search.best_params_}")

print("\n" + "="*80)
print("RELATÓRIO DE MELHORES HIPERPARÂMETROS POR BIOMA:")
display(pd.DataFrame(df_best_params))
print("="*80)

Iniciando Otimização de Hiperparâmetros (GridSearch) no Random Forest...
  ✅ Otimização e previsão concluída para: Amazônia | Best Params: {'max_depth': 5, 'n_estimators': 50, 'random_state': 42}
  ✅ Otimização e previsão concluída para: Caatinga | Best Params: {'max_depth': 5, 'n_estimators': 50, 'random_state': 42}
  ✅ Otimização e previsão concluída para: Cerrado | Best Params: {'max_depth': 5, 'n_estimators': 200, 'random_state': 42}
  ✅ Otimização e previsão concluída para: Mata Atlântica | Best Params: {'max_depth': 5, 'n_estimators': 50, 'random_state': 42}
  ✅ Otimização e previsão concluída para: Pampa | Best Params: {'max_depth': 5, 'n_estimators': 200, 'random_state': 42}
  ✅ Otimização e previsão concluída para: Pantanal | Best Params: {'max_depth': 5, 'n_estimators': 50, 'random_state': 42}

RELATÓRIO DE MELHORES HIPERPARÂMETROS POR BIOMA:


,bioma,melhores_params
0,Amazônia,"{'max_depth': 5, 'n_estimators': 50, 'random_s..."
1,Caatinga,"{'max_depth': 5, 'n_estimators': 50, 'random_s..."
2,Cerrado,"{'max_depth': 5, 'n_estimators': 200, 'random_..."
3,Mata Atlântica,"{'max_depth': 5, 'n_estimators': 50, 'random_s..."
4,Pampa,"{'max_depth': 5, 'n_estimators': 200, 'random_..."
5,Pantanal,"{'max_depth': 5, 'n_estimators': 50, 'random_s..."


# Avaliação Final e Comparação

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
from IPython.display import display # Necessário para exibir o DataFrame formatado

# A coluna TARGET_COL (nome da área desmatada) deve estar definida!
TARGET_COL = 'desmatado_anual_total_km2'

# 1. Preparação dos Dados de Previsão
# Valores Reais no Conjunto de Teste (y_true)
y_true = df_rf_otimizado_preds[TARGET_COL]

# Previsões do Random Forest Otimizado (obtidas no Grid Search)
y_pred_rf_otimizado = df_rf_otimizado_preds['rf_otimizado_pred']

# --- 2. Cálculo das Métricas do Random Forest Otimizado ---
rmse_rf_otimizado = np.sqrt(mean_squared_error(y_true, y_pred_rf_otimizado))
mae_rf_otimizado = mean_absolute_error(y_true, y_pred_rf_otimizado)
r2_rf_otimizado = r2_score(y_true, y_pred_rf_otimizado)

# --- 3. Geração e Exibição do Relatório Final de Comparação ---
print("\n" + "="*80)
print("           RELATÓRIO FINAL DE COMPARAÇÃO DE MODELOS (NO CONJUNTO DE TESTE)           ")
print("="*80)

# Estrutura a tabela com todas as métricas calculadas anteriormente (necessitam estar definidas!)
dados_comparacao_final = {
    'Métrica': ['RMSE (km²)', 'MAE (km²)', 'R² Score'],
    'Baseline (Ingênuo)': [rmse_baseline, mae_baseline, r2_baseline],
    'Regressão Linear': [rmse_regressao, mae_regressao, r2_regressao],
    'Random Forest (Inicial)': [rmse_rf, mae_rf, r2_rf],
    'Random Forest (Otimizado)': [rmse_rf_otimizado, mae_rf_otimizado, r2_rf_otimizado]
}

df_comparacao_final = pd.DataFrame(dados_comparacao_final)
df_comparacao_final = df_comparacao_final.set_index('Métrica').round(4)
display(df_comparacao_final)

print("\n" + "="*80)

print("\n" + "="*80)


           RELATÓRIO FINAL DE COMPARAÇÃO DE MODELOS (NO CONJUNTO DE TESTE)           


,Baseline (Ingênuo),Regressão Linear,Random Forest (Inicial),Random Forest (Otimizado)
Métrica,,,,
RMSE (km²),13529.5380,17671.0368,16406.8936,16034.7229
MAE (km²),8840.9333,14019.2924,11025.1049,10732.0265
R² Score,0.9986,0.9977,0.9980,0.9981


# Conclusão:

Embora o Modelo Baseline tenha obtido o menor erro, isto é um reflexo de uma pequena variação nos valores do período de teste e da simplicidade do modelo univariado. O objetivo do MVP era a comparação de abordagens, e a conclusão é que, para séries temporais voláteis, a simples tendência anual (Regressão Linear) não foi eficaz. O modelo Random Forest, embora não tenha superado o Baseline neste período, demonstrou ser a abordagem de ML mais robusta, atingindo um R2
  de 0.9981, o mais próximo do Baseline entre os modelos de ML, e seria o modelo preferido se tivéssemos que incluir variáveis externas (ex: preço de commodities) para aumentar a precisão."